## MLPClassifier model with RandomizedSearchCV to the best paramater

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import uniform

# Load the dataset
file_path = "cyberdata_clean.csv"
cyberdata = pd.read_csv(file_path)

if "Attack Type" in cyberdata.columns:
    
    # Define column lists by type
    nominal_cols = [
        "Protocol", "Packet Type", "Traffic Type", "Attack Signature",
        "Action Taken", "Network Segment", "Log Source", "City", "Region", "Browser", "Operating System"
    ]
    
    ordinal_cols = ["Severity Level", "Anomaly Score Category", "Packet Length Category"]
    numeric_cols = ["Source IP FirstOctet", "Destination IP FirstOctet"]
    
    # Define the complete list of feature columns to be used for training
    feature_cols = nominal_cols + ordinal_cols + numeric_cols
    
    # Create the ColumnTransformer with appropriate encoders
    preprocessor = ColumnTransformer(
        transformers=[
            ('nom', OneHotEncoder(handle_unknown='ignore'), nominal_cols),
            ('ord', OrdinalEncoder(), ordinal_cols),
            ('num', StandardScaler(), numeric_cols)
        ]
    )
    
    # Create a pipeline that applies the preprocessor and then the classifier
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('mlp', MLPClassifier(max_iter=1000, random_state=42))
    ])
    
    # Prepare features and target
    target = cyberdata["Attack Type"]
    features = cyberdata[feature_cols]

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.2, random_state=42, stratify=target
    )
    
    # Modified parameter distribution: using adam solver and lower regularization strength.
    param_dist = {
        'mlp__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
        'mlp__activation': ['relu', 'tanh'],         
        'mlp__solver': ['adam'],                       # adam for better convergence
        'mlp__alpha': uniform(0.0001, 0.01),           # lower regularization strength
        'mlp__learning_rate': ['constant'],            # learning rate strategy
        'mlp__learning_rate_init': uniform(0.001, 0.01)  # lower learning rate initialization
    }

    # Setup RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_dist,
        n_iter=20,
        cv=5,
        scoring='accuracy',
        random_state=42,
        n_jobs=-1
    )

    # Fit the pipeline with randomized search
    random_search.fit(X_train, y_train)

    print("Best hyperparameters:")
    print(random_search.best_params_)

    # Evaluate on the test set
    best_pipeline = random_search.best_estimator_
    y_pred = best_pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)

    print("Accuracy Score:", accuracy)
    print("Classification Report:\n", classification_rep)

else:
    print("Error: 'Attack Type' does not exist in the dataset")


Best hyperparameters:
{'mlp__activation': 'tanh', 'mlp__alpha': np.float64(0.005777003278199915), 'mlp__hidden_layer_sizes': (50,), 'mlp__learning_rate': 'constant', 'mlp__learning_rate_init': np.float64(0.010695846277645586), 'mlp__solver': 'adam'}
Accuracy Score: 0.327
Classification Report:
               precision    recall  f1-score   support

        DDoS       0.33      0.28      0.30      2686
   Intrusion       0.33      0.33      0.33      2653
     Malware       0.32      0.37      0.34      2661

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000



Persisting the model for deployment 


In [10]:
import joblib
# Save the entire pipeline for deployment ( we use pipeline to conserve the same scaler when a new data is entered)
joblib.dump(best_pipeline, "cyberAttackPipeline.joblib")


['cyberAttackPipeline.joblib']

Load the model

In [ ]:
import joblib
best_pipeline = joblib.load("cyberAttackPipeline.joblib")

Predicting

In [2]:
# index = X_test.index[1] ## second element in x_test matrix (from previous model testing)
# test_index =  X_test.loc[[index]]
# print(test_index)
# predictions = best_pipeline.predict(test_index)
# print(predictions)


In [1]:
# Choose two different indices
# index1 = X_test.index[1]
# index2 = X_test.index[15]

# Extract the rows as DataFrames
# row1 = X_test.loc[[index1]]
# row2 = X_test.loc[[index2]]

# Print the raw rows
# print("Row 1:")
# print(row1)
# print("\nRow 2:")
# print(row2)

# Get the preprocessed feature vectors
# preproc = best_pipeline.named_steps["preprocessor"]
# transformed_row1 = preproc.transform(row1)
# transformed_row2 = preproc.transform(row2)

# print("\nTransformed Row 1:")
# print(transformed_row1)
# print("\nTransformed Row 2:")
# print(transformed_row2)

# Get predictions
# pred1 = best_pipeline.predict(row1)
# pred2 = best_pipeline.predict(row2)

# print("\nPrediction for Row 1:", pred1)
# print("Prediction for Row 2:", pred2)
